# Chapter 26: Coroutines and the Event Loop

This notebook covers the foundations of asynchronous programming in Python: `async`/`await` syntax, coroutines, the event loop, and `asyncio.run()`. These are the building blocks for writing concurrent, non-blocking Python code.

## Key Concepts
- **Coroutines**: Functions defined with `async def` that can be suspended and resumed
- **`await`**: Yields control back to the event loop until a result is ready
- **Event loop**: The scheduler that runs coroutines and manages I/O
- **`asyncio.run()`**: The main entry point for running async code

## Section 1: What Is a Coroutine?

A coroutine is a function defined with `async def`. Calling it does **not** execute the body — it returns a coroutine object. The body only runs when the coroutine is awaited or scheduled on the event loop.

In [ ]:
import asyncio


# Define a coroutine with async def
async def greet() -> str:
    return "hello"


# Calling the coroutine function returns a coroutine object
coro = greet()
print(f"Type: {type(coro)}")
print(f"Is coroutine: {asyncio.iscoroutine(coro)}")

# The coroutine must be run to get the result
result = asyncio.run(coro)
print(f"Result: {result}")

In [ ]:
# Coroutine objects vs coroutine functions
async def say_hi() -> str:
    return "hi"


# The function itself is a coroutine function
print(f"Is coroutine function: {asyncio.iscoroutinefunction(say_hi)}")

# Calling it produces a coroutine object
coro_obj = say_hi()
print(f"Is coroutine object: {asyncio.iscoroutine(coro_obj)}")

# Always run the coroutine to avoid ResourceWarning
print(f"Result: {asyncio.run(coro_obj)}")

## Section 2: The `await` Keyword

`await` suspends the current coroutine until the awaited coroutine completes. This is how coroutines chain together — one coroutine can call another using `await`.

In [ ]:
# await chains coroutines together
async def add(a: int, b: int) -> int:
    """A simple async addition."""
    return a + b


async def main() -> int:
    """Main coroutine that awaits another."""
    result = await add(3, 4)
    return result


print(f"3 + 4 = {asyncio.run(main())}")

In [ ]:
# Multiple awaits in sequence
async def step_one() -> str:
    return "fetched data"


async def step_two(data: str) -> str:
    return f"processed {data}"


async def step_three(data: str) -> str:
    return f"saved {data}"


async def pipeline() -> str:
    """Sequential async pipeline."""
    raw = await step_one()
    processed = await step_two(raw)
    result = await step_three(processed)
    return result


print(asyncio.run(pipeline()))

## Section 3: `asyncio.sleep()` and Yielding Control

`asyncio.sleep()` is the async equivalent of `time.sleep()`. Unlike `time.sleep()`, it does **not** block the event loop — it suspends the current coroutine and allows other coroutines to run.

In [ ]:
import time


async def delayed_value() -> str:
    """Simulate an async operation with sleep."""
    await asyncio.sleep(0)  # Yield control, resume immediately
    return "done"


result = asyncio.run(delayed_value())
print(f"Result: {result}")

In [ ]:
# Demonstrating non-blocking sleep with timing
async def task_a() -> str:
    print("Task A: starting")
    await asyncio.sleep(0.1)
    print("Task A: finished")
    return "A"


async def task_b() -> str:
    print("Task B: starting")
    await asyncio.sleep(0.05)
    print("Task B: finished")
    return "B"


async def run_both() -> None:
    """Run both tasks concurrently — they interleave."""
    start = time.perf_counter()
    results = await asyncio.gather(task_a(), task_b())
    elapsed = time.perf_counter() - start
    print(f"\nResults: {results}")
    print(f"Elapsed: {elapsed:.3f}s (concurrent, not 0.15s)")


asyncio.run(run_both())

## Section 4: `asyncio.run()` — The Entry Point

`asyncio.run()` creates an event loop, runs the given coroutine until completion, and then closes the loop. It is the recommended way to start async code from synchronous code.

In [ ]:
# asyncio.run() handles the event loop lifecycle
async def compute_squares(numbers: list[int]) -> list[int]:
    """Compute squares asynchronously."""
    results: list[int] = []
    for n in numbers:
        await asyncio.sleep(0)  # Simulate async work
        results.append(n ** 2)
    return results


# asyncio.run() creates and manages the event loop
squares = asyncio.run(compute_squares([1, 2, 3, 4, 5]))
print(f"Squares: {squares}")

In [ ]:
# asyncio.run() can only be called when no event loop is already running
# This is important in Jupyter notebooks, which have their own event loop
# In scripts, asyncio.run() is the standard entry point:
#
#   async def main() -> None:
#       ...
#
#   if __name__ == "__main__":
#       asyncio.run(main())


async def main() -> str:
    """Standard async main pattern."""
    greeting = await greet()
    return f"The greeting is: {greeting}"


print(asyncio.run(main()))

## Section 5: The Event Loop Under the Hood

The event loop is the core of `asyncio`. It keeps track of all running coroutines and switches between them at `await` points. You rarely interact with the loop directly — `asyncio.run()` manages it for you.

In [ ]:
# Observing the event loop from inside a coroutine
async def show_loop_info() -> None:
    loop = asyncio.get_running_loop()
    print(f"Loop type: {type(loop).__name__}")
    print(f"Loop running: {loop.is_running()}")
    print(f"Loop closed: {loop.is_closed()}")


asyncio.run(show_loop_info())

In [ ]:
# Understanding execution order with the event loop
async def announce(label: str, delay: float) -> str:
    print(f"  [{label}] starting")
    await asyncio.sleep(delay)
    print(f"  [{label}] done after {delay}s")
    return label


async def demonstrate_order() -> None:
    """Show that the event loop interleaves coroutines."""
    print("Launching three coroutines concurrently:")
    results = await asyncio.gather(
        announce("slow", 0.15),
        announce("medium", 0.10),
        announce("fast", 0.05),
    )
    print(f"\nResults (in original order): {results}")


asyncio.run(demonstrate_order())

## Section 6: Coroutines vs Regular Functions

Understanding when to use `async def` vs `def` is important. Coroutines are best for I/O-bound work — network calls, file I/O, database queries. CPU-bound work should use threads or processes instead.

In [ ]:
# Regular function - blocks the thread
def sync_fetch(url: str) -> str:
    """Simulates a blocking I/O call."""
    time.sleep(0.05)  # Blocks the entire thread
    return f"data from {url}"


# Async coroutine - yields control during I/O
async def async_fetch(url: str) -> str:
    """Simulates a non-blocking I/O call."""
    await asyncio.sleep(0.05)  # Yields control to the loop
    return f"data from {url}"


# Compare sequential sync vs concurrent async
urls = ["api/users", "api/products", "api/orders"]

# Synchronous: runs one after another
start = time.perf_counter()
sync_results = [sync_fetch(url) for url in urls]
sync_elapsed = time.perf_counter() - start
print(f"Sync results: {sync_results}")
print(f"Sync elapsed: {sync_elapsed:.3f}s")


# Asynchronous: runs concurrently
async def fetch_all(urls: list[str]) -> list[str]:
    return await asyncio.gather(*(async_fetch(url) for url in urls))


start = time.perf_counter()
async_results = asyncio.run(fetch_all(urls))
async_elapsed = time.perf_counter() - start
print(f"\nAsync results: {async_results}")
print(f"Async elapsed: {async_elapsed:.3f}s")
print(f"\nSpeedup: {sync_elapsed / async_elapsed:.1f}x")

## Section 7: Awaitable Objects

Any object that implements the `__await__` method is awaitable. Coroutines, Tasks, and Futures are all awaitables. You can create your own awaitable objects too.

In [ ]:
from collections.abc import Generator


# Custom awaitable object
class DelayedResult:
    """An awaitable that yields a result after a delay."""

    def __init__(self, value: str, delay: float) -> None:
        self.value = value
        self.delay = delay

    def __await__(self) -> Generator[None, None, str]:
        """Make this object awaitable."""
        yield from asyncio.sleep(self.delay).__await__()
        return self.value


async def use_custom_awaitable() -> None:
    result = await DelayedResult("custom result", 0.01)
    print(f"Got: {result}")


asyncio.run(use_custom_awaitable())

In [ ]:
# Checking if something is awaitable
import inspect


async def example_coro() -> int:
    return 42


coro = example_coro()
print(f"Coroutine is awaitable: {inspect.isawaitable(coro)}")
print(f"Regular int is awaitable: {inspect.isawaitable(42)}")
print(f"String is awaitable: {inspect.isawaitable('hello')}")

# Clean up the coroutine
asyncio.run(coro)

## Section 8: Error Handling in Coroutines

Exceptions in coroutines work the same as in regular functions. An exception raised inside a coroutine propagates to the caller when the coroutine is awaited.

In [ ]:
# Exceptions propagate through await
async def risky_operation() -> str:
    await asyncio.sleep(0)
    raise ValueError("something went wrong")


async def safe_caller() -> str:
    """Handle errors from awaited coroutines."""
    try:
        result = await risky_operation()
        return result
    except ValueError as e:
        print(f"Caught error: {e}")
        return "fallback value"


result = asyncio.run(safe_caller())
print(f"Final result: {result}")

In [ ]:
# Handling errors with asyncio.gather using return_exceptions
async def succeed() -> str:
    return "success"


async def fail() -> str:
    raise RuntimeError("failure")


async def mixed_results() -> None:
    """Gather with return_exceptions=True to capture errors."""
    results = await asyncio.gather(
        succeed(),
        fail(),
        succeed(),
        return_exceptions=True,
    )
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"  Task {i}: ERROR - {result}")
        else:
            print(f"  Task {i}: OK - {result}")


asyncio.run(mixed_results())

## Summary

### Coroutines
- Defined with `async def`, return coroutine objects when called
- Must be awaited or scheduled on the event loop to execute
- Use `asyncio.iscoroutine()` and `asyncio.iscoroutinefunction()` for inspection

### The `await` Keyword
- Suspends the current coroutine until the awaited result is ready
- Only valid inside `async def` functions
- Chains coroutines together in a sequential pipeline

### The Event Loop
- Schedules and runs coroutines, switching at `await` points
- Enables concurrency without threads
- Access with `asyncio.get_running_loop()` from inside a coroutine

### `asyncio.run()`
- Creates an event loop, runs the coroutine, and closes the loop
- The standard entry point for async programs
- Cannot be called when another event loop is already running

### Key Patterns
- Use `asyncio.sleep()` instead of `time.sleep()` in async code
- Handle exceptions with `try`/`except` around `await` expressions
- Async is best for I/O-bound tasks, not CPU-bound computation